### Machine Learning-based Data Analysis with Case Study
### [Practice 4] : Anime Recommendation System
    1. 문제정의하기(Problem Define)
    2. 라이브러리 불러오기(Libraries Setting)
    3. 데이터 수집하기(Data Collection)
    4. 데이터 탐색하기(Data Exploration)
    5. 전처리하기(Preprocessing)
    6. 모델링하기(Modeling)
        - 6.1 품목 추천(Item based Recommendations)
        - 6.2 사용자에게 개인맞춤 추천하기 (Personalized Recommendations)

#### [1]: Problem Define : 문제정의
- 애니메이션 추천 시스템 - 협업 필터링 Matrix Factorization : 애니메이션을 추천합니다.

- Data: MyAnimeList Dataset
     - y = f(x)
     - y: 애니메이션 추천
     - x: 애니메이션 데이터, 사용자 데이터

#### [2] 라이브러리 불러오기(Libraries Setting)

In [ ]:
import numpy as np # Numeric Python 
import pandas as pd # Data Processing and Database
import matplotlib.pyplot as plt # Visualization
import seaborn as sns # Visualization

from sklearn.decomposition import TruncatedSVD # Recommendations
from scipy.sparse.linalg import svds # Recommendations
import pickle

#### [3] 데이터 수집하기(Data Collection)

In [ ]:
# 평점 데이터 샘플링 (전체 데이터가 너무 크므로 일부만 사용)
rating_data = pd.read_csv('./data/rating_complete.csv', nrows=100000)
anime_data = pd.read_csv('./data/anime_with_synopsis.csv')
rating_data.head()

해석) 
- user_id가 같다는 의미는 한 사람임을 알려줍니다. 
- 한 사람이 여러 애니메이션을 볼 수 있습니다. 
- 한 사람이 여러 애니메이션들에 대해 점수를 매길 수 있습니다.

In [ ]:
anime_data.head()

#### [4] 데이터 탐색하기(Data Exploration)

### [데이터 셋 구조]

In [ ]:
print(anime_data.shape)
print(rating_data.shape)

### [데이터 타입]

In [ ]:
anime_data.info()

### [데이터 타입]

In [ ]:
rating_data.info()

### [데이터 통계]

In [ ]:
anime_data.describe()

### [데이터 통계]

In [ ]:
rating_data.describe()

In [ ]:
rating_data.head(10)

In [ ]:
userId_duplicate = rating_data.drop_duplicates(['user_id'])
userId_duplicate.head()

In [ ]:
userId_duplicate.shape

해석) 사용자 수를 확인합니다.

#### [5] 전처리하기(Preprocessing)

In [ ]:
# -1 평점 제거 (평가하지 않은 경우)
rating_data = rating_data[rating_data['rating'] != -1]
rating_data.head()

In [ ]:
# 애니메이션 정보에서 필요한 컬럼만 선택
anime_data_clean = anime_data[['MAL_ID', 'Name', 'Score', 'Genres']].copy()
anime_data_clean = anime_data_clean.rename(columns={'MAL_ID': 'anime_id'})
anime_data_clean.head()

In [ ]:
# 데이터 샘플링 (효율성)
# 평점이 많은 인기 애니메이션
anime_rating_counts = rating_data['anime_id'].value_counts()
popular_animes = anime_rating_counts[anime_rating_counts >= 50].index

# 활성 사용자
user_rating_counts = rating_data['user_id'].value_counts()
active_users = user_rating_counts[user_rating_counts >= 20].index

# 필터링
rating_data_sampled = rating_data[
    (rating_data['anime_id'].isin(popular_animes)) &
    (rating_data['user_id'].isin(active_users))
]

print(f"Sampled Data Shape: {rating_data_sampled.shape}")
print(f"Users: {rating_data_sampled['user_id'].nunique()}")
print(f"Animes: {rating_data_sampled['anime_id'].nunique()}")

In [ ]:
user_anime_rating = rating_data_sampled.pivot_table('rating', index='user_id', columns='anime_id').fillna(0)
user_anime_rating.shape

In [ ]:
user_anime_rating.head()

## 행이 애니메이션, 열이 사용자

In [ ]:
anime_user_rating = user_anime_rating.values.T
anime_user_rating.shape

#### [6] 모델링하기(Modeling)

#### [6.1] 품목 추천(Item based Recommendations)

In [ ]:
SVD = TruncatedSVD(n_components=12)
matrix = SVD.fit_transform(anime_user_rating)
matrix.shape

In [ ]:
matrix[0]

해석) 12개의 component로 차원을 축소했습니다.

In [ ]:
corr = np.corrcoef(matrix)
corr.shape

[알아두기] 피어슨 상관계수
- np.corrcoef(): 피어슨 상관계수 값을 계산합니다.
- 애니메이션-애니메이션 간의 상관관계를 봅니다.

In [ ]:
anime_title = user_anime_rating.columns
anime_title_list = list(anime_title)
sample_anime = anime_title_list[0]
sample_index = 0
sample_index

In [ ]:
corr_sample = corr[sample_index]
list(anime_title[(corr_sample >= 0.9)])[:10]

해석) 샘플 애니메이션과 유사한 작품 10개를 추천합니다.

#### [6.2] 사용자에게 개인맞춤 추천하기 (Personalized Recommendations)

In [ ]:
df_ratings = rating_data_sampled.copy()
df_animes = anime_data_clean.copy()
df_ratings.head()

In [ ]:
df_animes.head()

In [ ]:
df_user_anime_ratings = df_ratings.pivot_table(values='rating', index='user_id', columns='anime_id').fillna(0)
df_user_anime_ratings.shape

In [ ]:
df_user_anime_ratings.head()

In [ ]:
matrix = df_user_anime_ratings.to_numpy()
user_ratings_mean = np.mean(matrix, axis=1)
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)
matrix.shape

In [ ]:
user_ratings_mean.shape

In [ ]:
pd.DataFrame(matrix_user_mean, columns=df_user_anime_ratings.columns).head()

In [ ]:
U, sigma, Vt = svds(matrix_user_mean, k=12)

In [ ]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

In [ ]:
sigma = np.diag(sigma)
sigma.shape

In [ ]:
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns=df_user_anime_ratings.columns, index=df_user_anime_ratings.index)
df_svd_preds.head()

In [ ]:
df_svd_preds.shape

#### 추천 함수 만들기

In [ ]:
def recommend_animes(df_svd_preds, user_id, ori_animes_df, ori_ratings_df, num_recommendations):
    if user_id not in df_svd_preds.index:
        print(f"사용자 ID {user_id}가 존재하지 않습니다.")
        return None, None
    
    sorted_user_predictions = df_svd_preds.loc[user_id].sort_values(ascending=False)
    user_data = ori_ratings_df[ori_ratings_df['user_id'] == user_id]
    user_history = user_data.merge(ori_animes_df, on='anime_id').sort_values(['rating'], ascending=False)
    recommendations = ori_animes_df[~ori_animes_df['anime_id'].isin(user_history['anime_id'])]
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on='anime_id')
    recommendations = recommendations.rename(columns={user_id: 'Predictions'}).sort_values('Predictions', ascending=False).iloc[:num_recommendations, :]
    
    return user_history, recommendations

In [ ]:
sample_user_id = df_user_anime_ratings.index[0]
already_rated, predictions = recommend_animes(df_svd_preds, sample_user_id, df_animes, df_ratings, 10)

In [ ]:
already_rated.head(10)

In [ ]:
predictions

#### 결론)
- 사용자 별로 다르게 추천됨을 알수 있습니다.
- 사용자 맞춤 추천을 할 수 있습니다.